In [ ]:
# default_exp simulators.simulator_mixins

# SimulatorMixins

> A mixin for simulators to implement gate application defined in a given `circuit` and optional `fault_circuit`.

In [ ]:
# hide
from nbdev.showdoc import *

The `CircuitRunner` mixin defines two methods `_apply_gate` and `run`, of which only the latter `run` method exposes the main functionality of a simulator. 

In [ ]:
#export
class CircuitRunnerMixin:
    """Simulator mixin for running quantum circuits"""
        
    def _apply_gate(self, gate_symbol, qubits):
        """Apply a gate to the `qubits` of the current state."""
        
        gate = getattr(self, gate_symbolol)    
        args = (qubits,) if type(qubits)==int else qubits
        return gate(*args)
    
    def run(self, circuit, fault_circuit=None):
        """Apply gates in `circuit` sequentially to current state.
        If `fault_circuit` is specified apply fault gates at end of each tick.
        
        Measurement results are saves in the order in which they appear in the
        circuit into an output string."""
        
        msmt_res = []
        for tick_index in range(circuit.n_ticks):
            
            for gate, qubits in circuit[tick_index].items():
                for qubit in qubits:
                    res = self._apply_gate(gate, qubit)

                    if res is not None:
                        msmt_res.append( int(res.value) )
            if fault_circuit:
                for f_gate, f_qubits in fault_circuit[tick_index].items():
                    for f_qubit in f_qubits:
                        self._apply_gate(f_gate, f_qubit)

        if msmt_res: 
            return ''.join(map(str, msmt_res))
        else: 
            return None # no measurement

This mixin is not meant to be used as a standalone object. It is rather meant to be referenced (i.e. inherited) by a `Simulator` class to implement the *circuit-running* functionality.

In [ ]:
from qsam.circuit import Circuit
from qsam.simulators.chp import MeasureResult

class TestSim(CircuitRunnerMixin):
    def X(self, i):
        pass
    def measure(self, i):
        if i == 0 or i == 2:
            return MeasureResult(value=True, determined=True)
        elif i == 1:
            return MeasureResult(value=False, determined=True)
    
circuit = Circuit([{'X': [1,2,3]}, {'measure': [2,1]}, {'measure': [0]}])

test_sim = TestSim()
res = test_sim.run(circuit) # TestSim knows how to run() a circuit
print(res)

101


The circuit run returns a string of measurement results. It is important to note that the order of bits in the measurement output follows the order of measurements defined in the circuit **exactly**. We can for example see that the first digit corresponds to the Z-measurement on qubit number 2, as this measurement is defined before the one on qubit number 1 in the same tick.